In [3]:
from Bio.PDB import PDBParser, Superimposer, PPBuilder, PDBIO
from Bio import pairwise2
import py3Dmol

### Bloque 6 – Comparación con una estructura experimental

Con el objetivo de evaluar la precisión de la predicción estructural generada por AlphaFold, se comparó el
modelo predicho con una estructura experimental de la beta-2-microglobulina disponible en el Protein Data
Bank (PDB). Ambas estructuras se alinearon utilizando los átomos Cα de los residuos equivalentes y se calculó
el RMSD como medida de similitud estructural global

In [2]:
af_path = "data/AF-P61769-F1-model_v6.pdb"
exp_path = "data/2D4F.pdb"   

parser = PDBParser(QUIET=True)
ppb = PPBuilder()

af_struct = parser.get_structure("AF", af_path)
exp_struct = parser.get_structure("EXP", exp_path)

af_chain = list(af_struct[0].get_chains())[0]
exp_chain = list(exp_struct[0].get_chains())[0]

af_seq = str(ppb.build_peptides(af_chain)[0].get_sequence())
exp_seq = str(ppb.build_peptides(exp_chain)[0].get_sequence())

aln = pairwise2.align.globalxx(af_seq, exp_seq, one_alignment_only=True)[0]
af_aln, exp_aln = aln.seqA, aln.seqB

af_residues = [res for res in af_chain if res.id[0] == " " and "CA" in res]
exp_residues = [res for res in exp_chain if res.id[0] == " " and "CA" in res]

af_idx = 0
exp_idx = 0

af_ca_atoms = []
exp_ca_atoms = []

for a, b in zip(af_aln, exp_aln):
    if a != "-" and b != "-":
        # Emparejar Cα
        af_ca_atoms.append(af_residues[af_idx]["CA"])
        exp_ca_atoms.append(exp_residues[exp_idx]["CA"])
    if a != "-":
        af_idx += 1
    if b != "-":
        exp_idx += 1
        
sup = Superimposer()
sup.set_atoms(exp_ca_atoms, af_ca_atoms)  # alinea AlphaFold sobre el experimental
rmsd = sup.rms

sup.apply(list(af_struct.get_atoms()))

print("Número de Cα emparejados:", len(af_ca_atoms))
print(f"RMSD (Cα) AlphaFold vs experimental: {rmsd:.3f} Å")

Número de Cα emparejados: 98
RMSD (Cα) AlphaFold vs experimental: 4.350 Å


El alineamiento permitió emparejar un total de 98 átomos Cα, lo que representa una fracción significativa de
la secuencia total de la proteína. El valor de RMSD obtenido fue de **4.350 Å**, lo que indica una concordancia
global moderada entre el modelo predicho y la estructura experimental.

In [4]:
io = PDBIO()
io.set_structure(af_struct)
io.save("data/AF_aligned_to_exp.pdb")

# Leer ambos modelos para visualizar
with open("data/AF_aligned_to_exp.pdb") as f:
    af_pdb = f.read()

with open(exp_path) as f:
    exp_pdb = f.read()

view = py3Dmol.view(width=900, height=500)

# Experimental (gris)
view.addModel(exp_pdb, "pdb")
view.setStyle({"model": 0}, {"cartoon": {"color": "lightgray"}})

# AlphaFold alineado (azul)
view.addModel(af_pdb, "pdb")
view.setStyle({"model": 1}, {"cartoon": {"color": "steelblue"}})

view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

La visualización superpuesta de ambas estructuras muestra que el núcleo estructural de la proteína, dominado
por láminas β características del plegamiento tipo inmunoglobulina, presenta una buena coincidencia espacial.
Sin embargo, se observan diferencias más acusadas en regiones periféricas, en particular en la región
helicoidal alargada, que adopta orientaciones distintas en ambos modelos.

Estas discrepancias pueden atribuirse a la mayor flexibilidad conformacional de dichas regiones, así como a
diferencias en las condiciones experimentales bajo las cuales se resolvió la estructura experimental. En
conjunto, los resultados indican que AlphaFold reproduce correctamente la arquitectura global de la
beta-2-microglobulina, aunque existen variaciones locales que afectan al valor global de RMSD.